## YouTube Video → AI Study Notes

In [35]:
import re
import torch
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer

In [36]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [56]:
def extract_video_id(url):
    """Extracts video ID from different YouTube URL formats."""
    # We use Regex to hunt for the 11-character ID after 'v=' or 'youtu.be/'
    match = re.search(r"(?:v=|youtu\.be/)([a-zA-Z0-9_-]{11})", url)
    return match.group(1) if match else None

def get_transcript(video_id):
    """Fetch transcript using the NEW API format."""
    try:
        api = YouTubeTranscriptApi()
        # The .fetch method grabs the subtitle object list
        transcript = api.fetch(video_id)
        # We join the list into a single long string of text
        return " ".join([t.text for t in transcript])

    except TranscriptsDisabled:
        return "Error: Transcripts are disabled for this video."
    except NoTranscriptFound:
        return "Error: No transcript found for this video."
    except Exception as e:
        return f"Error: {str(e)}"

In [63]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Check if we have a GPU (CUDA) available to speed things up
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/flan-t5-base"

# Load the tokenizer (translates text to numbers)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load the model (the neural network) and move it to the GPU/CPU
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Loading weights: 100%|██████████| 282/282 [00:00<00:00, 493.98it/s, Materializing param=shared.weight]                                                       
The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [64]:
def summarize_chunk(text_chunk):
    """
    Summarizes a text chunk with enhanced settings for longer, more informative summaries.
    """
    # Prompt for clear, detailed summary
    prompt = f"Summarize the following text clearly and in detail:\n{text_chunk}"

    # Tokenize input
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(device)

    # Generate summary with enhanced parameters
    summary_ids = model.generate(
        **inputs,
        max_new_tokens=300,        # Allow longer summaries
        num_beams=6,               # Explore more paths for higher quality
        length_penalty=1.2,        # Favor longer outputs
        min_length=100,            # Ensure minimum content
        no_repeat_ngram_size=3,    # Reduce repetition
        early_stopping=True,
        temperature=0.7,           # Slightly creative output
        top_p=0.9                  # Nucleus sampling
    )

    # Decode summary back to text
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [65]:
def chunk_text(text, chunk_size=1200):
    sentences = text.split(". ")
    chunks, current_chunk = [], ""

    for sentence in sentences:
        # Check if adding the next sentence exceeds our limit
        if len(current_chunk) + len(sentence) < chunk_size:
            current_chunk += sentence + ". "
        else:
            # If full, seal the chunk and start a new one
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [72]:
def generate_video_notes(video_url):
    print(f"\n🎬 Processing video: {video_url}")

    video_id = extract_video_id(video_url)
    if not video_id:
        print("Invalid YouTube URL.")
        return

    print("🎧 Fetching transcript...")
    transcript = get_transcript(video_id)

    if transcript.startswith("Error"):
        print(transcript)
        return

    print("🔪 Chunking transcript...")
    chunks = chunk_text(transcript)
    print(f"   -> {len(chunks)} chunks created.")

    print("🧠 Generating AI notes...")
    notes = []

    # Loop through chunks and summarize each one
    for i, chunk in enumerate(chunks):
        print(f"   Summarizing chunk {i+1}/{len(chunks)}...")
        summary = summarize_chunk(chunk)
        notes.append(f"- {summary}")

    print("\n" + "="*50)
    print("📝 AI GENERATED NOTES")
    print("="*50)
    print("\n".join(notes))


if __name__ == "__main__":
    url = input("Paste YouTube URL: ")
    generate_video_notes(url)


🎬 Processing video: https://youtu.be/T-D1OfcDW1M?si=8mdhYF3WSHOtI3CQ
🎧 Fetching transcript...
🔪 Chunking transcript...
   -> 6 chunks created.
🧠 Generating AI notes...
   Summarizing chunk 1/6...
   Summarizing chunk 2/6...
   Summarizing chunk 3/6...
   Summarizing chunk 4/6...
   Summarizing chunk 5/6...
   Summarizing chunk 6/6...

📝 AI GENERATED NOTES
- Retrieval-Augmented Generation (RAG) is a framework to help large language models be more accurate and more up-to-date. This refers to large language modeling, or LLMs, that generate text in response to a user query, referred to as a prompt. These models can have some undesirable behavior. I want to tell you an anecdote to illustrate what I'm saying. So my kids, they recently asked me this question: "In our solar system, what planet has the most moons?" And I read an article and the article said that Jupiter and 88 moons. So that's really great that you're asking this question.
- I don't know. I don’t have a source. I'm out of date